In [2]:
pip install articledownloader


Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install Autologging


In [4]:
from bs4 import BeautifulSoup
import re
import unicodedata
from articledownloader.articledownloader import ArticleDownloader
downloader = ArticleDownloader('e46dd526a567506b5b2a4a6dda9963cd') # Personal Elsevier api key
import xml.etree.cElementTree as ET
import pandas as pd

In [5]:
import os, glob, re, unicodedata, xml.etree.ElementTree as ET

# ── PATH TO DOWNLOADED XMLs ───────────────────────────────
XML_DIR = os.path.join(os.path.dirname('venv'), "xmlfiles")  # adjust if needed
# ───────────────────────────────────────────────────────────

repo         = []   # [doi, title, abstract, [para1, para2, …], n_paras]
para_counter = 0

# namespaces used in Elsevier Full-Text XML
NS_BASIC = '{http://prismstandard.org/namespaces/basic/2.0/}'
NS_DC    = '{http://purl.org/dc/elements/1.1/}'
NS_PARA  = '{http://www.elsevier.com/xml/common/dtd}para'

for xml_path in glob.glob(os.path.join(XML_DIR, "*.xml")):
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        paper_detail = []

        # -- DOI, title, abstract (they live in the first child “head” element) --
        for elem in root[0]:
            if   elem.tag == NS_BASIC + "doi":
                paper_detail.append(elem.text or "")
            elif elem.tag == NS_DC + "title":
                paper_detail.append(elem.text or "")
            elif elem.tag == NS_DC + "description":
                txt = re.sub(r"\s+", " ", (elem.text or "").strip())
                paper_detail.append(unicodedata.normalize("NFKD", txt))

        # -- Paragraphs --
        paragraphs = []
        for p in tree.iter(NS_PARA):
            chunk = "".join(unicodedata.normalize("NFKD", t.strip())
                            for t in p.itertext()).strip()
            if chunk:
                paragraphs.append(chunk)

        paper_detail.append(paragraphs)
        paper_detail.append(len(paragraphs))

        para_counter += len(paragraphs)
        repo.append(paper_detail)

    except Exception as e:
        # Skip malformed / unexpected XML but keep going
        print(f"Skipped {os.path.basename(xml_path)}: {e}")

print(f"\nProcessed articles : {len(repo)}")
print(f"Total paragraphs   : {para_counter}")



Processed articles : 3014
Total paragraphs   : 110159


In [6]:
import pandas as pd

df = pd.DataFrame(repo,
                  columns=["doi", "title", "abstract", "paragraphs", "n_paras"])
# explode if you want one row per paragraph
df_long = df.explode("paragraphs", ignore_index=True)   # one row per para


In [7]:
df.to_pickle("article_meta.pkl")          # preserves Python objects
df_long.to_csv("article_paragraphs.csv", index=False)


In [8]:
import sys
from pathlib import Path
import pandas as pd

# df already exists in memory ────────────────────────────────
# If not, re-run the cell that builds it from repo first.

# 1) Locate the root of the *currently-active* venv
venv_root = Path(sys.prefix)            # e.g. …/myproject/venv

# 2) Decide where inside that venv you want the CSV
csv_path = venv_root / "xmlAndHTML_data.csv"   # …/venv/xmlAndHTML_data.csv

# (optional) put it in a subfolder instead:
# data_dir = venv_root / "data"
# data_dir.mkdir(exist_ok=True)
# csv_path = data_dir / "xmlAndHTML_data.csv"

# 3) Write the DataFrame
df.to_csv(csv_path, index=False)

print(f"✓  DataFrame saved to: {csv_path}")


✓  DataFrame saved to: c:\Users\hp\Desktop\feb research\venv\xmlAndHTML_data.csv


In [9]:
import ast, pandas as pd, sys
from pathlib import Path

csv_path = Path(sys.prefix) / "xmlAndHTML_data.csv"

xml = pd.read_csv(
        csv_path,
        converters={"Para_list": ast.literal_eval}
      )

# Make a fresh serial index starting at 0
xml.index = range(len(xml))
xml.index.name = "Serial_No"

xml.tail()   # check it looks right


,doi,title,abstract,paragraphs,n_paras
Serial_No,,,,,
3009,10.1016/S1466-6049(01)00122-2,Importance of soft solution processing for adv...,Barium zirconate is an interesting material fo...,['Ceramic materials are used for a wide variet...,16
3010,10.1016/S1468-6996(01)00120-6,Preparation of alumina–silica–nickel nanocompo...,Ceramic based composites with dispersion of na...,['Nanocomposite materials have generated profo...,19
3011,10.1016/S1468-6996(02)00038-4,Boride coatings on non-ferrous materials in a ...,Fluidized bed technology has been successfully...,['Heat treatments of alloys in fluidized bed r...,32
3012,10.1016/S1468-6996(03)00011-1,The preparation and properties of CeO2–TiO2 fi...,Ultraviolet absorbing CeO2–TiO2 coatings were ...,['It is well known that amorphous and crystall...,22
3013,10.1016/S2238-7854(12)70023-6,Use of Stainless Steel AISI 304 for Catalytic ...,Nanomaterials (CNMs) have extraordinary mechan...,[],0


In [10]:
import ast, sys, pandas as pd
from pathlib import Path

csv_path = Path(sys.prefix) / "xmlAndHTML_data.csv"   # …/venv/xmlAndHTML_data.csv
combo     = pd.read_csv(csv_path,
                        converters={"Para_list": ast.literal_eval})

combo.shape


(3014, 5)

In [ ]:
list_para = []
for para_list in df2:
    for para in para_list:
        list_para.append(para)
    
len(list_para)  

110159